In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import cvxpy as cp
import random

In [2]:
def NKron(*args):
  result = np.array([[1.0]])
  for op in args:
    result = np.kron(result, op)
  return result

In [3]:
Id = np.eye(2)
X = np.array([[0.0, 1.0],[1.0, 0.0]])
Z = np.array([[1.0, 0.0],[0.0, -1.0]])
Y = np.matmul(X,Z)

In [4]:
NormalizeState = lambda state: state / sp.linalg.norm(state)
zero = np.array([[1.0], [0.0]]) # |0>
one = np.array([[0.0], [1.0]]) # |1>

## Generators

In [5]:
def NKronModified(checkRowMod):
  result = np.array([[1.0]])
  for ind in checkRowMod:
    if(ind == 0):
        op = Id
    elif(ind == 1):
        op = X
    elif(ind == 2):
        op = Y
    elif(ind == 3):
        op = Z
    result = np.kron(result, op)
  return result

def getGenerator(checkRow):
    checkRowModified = np.zeros(n, dtype=int)
    
    checkRowModified[(checkRow[:n] == checkRow[n:]) & (checkRow[n:] == 1)] = 2
    checkRowModified[(checkRow[:n] == 1) & (checkRowModified != 2)] = 1
    checkRowModified[(checkRow[n:] == 1) & (checkRowModified != 2)] = 3
    
    return NKronModified(checkRowModified)    

In [6]:
comparingAccuracy_decoded = 1e-7
comparingAccuracy_encoded = 1e-5
comparingAccuracy_syndrome = 1e-5
comparingAccuracy_method = 1e-5

In [7]:
# change check matrix here

# checkMatrix = np.array([[1,1,1,1,1,1,1,1, 0,0,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0,0, 1,1,1,1,1,1,1,1],
#                         [0,1,0,1,1,0,1,0, 0,0,0,0,1,1,1,1],
#                         [0,1,0,1,0,1,0,1, 0,0,1,1,0,0,1,1],
#                         [0,1,1,0,1,0,0,1, 0,1,0,1,0,1,0,1]])

# checkMatrix = np.array([[0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 1,0,1,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,1,1,0,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,1,0,1,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,0,0,0,1,1,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,0,0,0,1,0,1],
#                         [1,1,1,1,1,1,0,0,0, 0,0,0,0,0,0,0,0,0],
#                         [1,1,1,0,0,0,1,1,1, 0,0,0,0,0,0,0,0,0]])

checkMatrix = np.array([[1,0,0,1,0, 0,1,1,0,0],
                        [0,1,0,0,1, 0,0,1,1,0],
                        [1,0,1,0,0, 0,0,0,1,1],
                        [0,1,0,1,0, 1,0,0,0,1]])

# checkMatrix = np.array([[0,0,0,1,1,1,1, 0,0,0,0,0,0,0],
#                         [0,1,1,0,0,1,1, 0,0,0,0,0,0,0],
#                         [1,0,1,0,1,0,1, 0,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0, 0,0,0,1,1,1,1],
#                         [0,0,0,0,0,0,0, 0,1,1,0,0,1,1],
#                         [0,0,0,0,0,0,0, 1,0,1,0,1,0,1]])

n = int(checkMatrix.shape[1]/2)
k = n-checkMatrix.shape[0]

gi = np.zeros([n-k, 2**n, 2**n])
for i in range(n-k):
    gi[i,:,:] = getGenerator(checkMatrix[i,:])

## G matrix

In [8]:
def NKron1DGeneral(ipArray):
    result = np.array([[1.0]])
    for i in ipArray:
        if(i==1):
            op = one
        elif(i==0):
            op = zero
        result = np.kron(result, op)
    return result

#### Get generator matrix G

In [9]:
Gmatrix = np.eye(gi[0,:,:].shape[0], gi[0,:,:].shape[1]) # generator matrix corresponding to this code
for i in range(n-k):
    Gmatrix = Gmatrix + np.matmul(gi[i,:,:], Gmatrix)
Gmatrix = np.round(Gmatrix)

#### Get non-zero and unique columns of G

In [10]:
# get boolean array if the columns are zero or not
zeroCols = np.zeros(Gmatrix.shape[1])
for i in range(Gmatrix.shape[1]):
    zeroCols[i] = all(Gmatrix[:,i] == np.zeros(Gmatrix.shape[0]))

# get indices of non-zero columns
nonZeroColsList = np.argwhere(zeroCols==0).flatten()

# get all non zero columns
GmatrixNonZero = np.zeros([Gmatrix.shape[0], nonZeroColsList.shape[0]])
i = 0
for ind in nonZeroColsList:
    GmatrixNonZero[:,i] = Gmatrix[:,ind]
    i = i+1

# get all non zero and unique columns and there indices
GmatrixNonZeroUniqueInd, nonZeroUniqueInd = np.unique(GmatrixNonZero, axis = 1, return_index=True)
nonZeroUniqueInd = nonZeroColsList[nonZeroUniqueInd]

In [11]:
print('Rank of G = ' + str(np.linalg.matrix_rank(Gmatrix)))
print('Shape of G = ' + str(Gmatrix.shape))

Rank of G = 2
Shape of G = (32, 32)


## Get Syndrome from Error

In [443]:
def getSyndromeFromError(channel_error):
    tx_qbits = np.ones(2**k)
    tx_qbits = NormalizeState(tx_qbits)

    # Convert qbits to tensor product format
    tx_decoded = np.zeros(2**n)
    # get extended qbits corresponding to non-zero column indices of G matrix
    i = 0
    for nonZeroIndex in np.sort(nonZeroUniqueInd):
        if(i>=2**k):
            break
        tx_decoded[nonZeroIndex] = tx_qbits[i]
        i = i+1
    tx_decoded = NormalizeState(tx_decoded)

    # encode transmit qbits
    tx_encoded = NormalizeState(tx_decoded) # encoded transmit qbits
    for i in range(n-k):
        tx_encoded = tx_encoded + np.matmul(gi[i,:,:], tx_encoded) # encode using generators
    tx_encoded = NormalizeState(tx_encoded) # encoded transmit qbits

    # channel
    rx_erry = np.dot(channel_error, tx_encoded) # received qbits with errors

    # syndrome check
    syndr = np.zeros([n-k, 1]) # syndrome
    for i in range(n-k):
        syndr[i] = np.dot(rx_erry.transpose(), np.dot(gi[i,:,:], rx_erry))
        
    syndr[syndr>0] = 0
    syndr[syndr<0] = 1
        
    return np.ndarray.astype(np.round(syndr), 'int').flatten()

## Get Error from Syndrome

In [444]:
# def getCardin(myVector):
#     return np.sum(myVector != 0)

# def getErrorFromSyndrome(syndr):
#     success = 0
#     finalError = np.zeros(2*errCheckRowModified.shape[0])
#     while(True):
#         maxMetric = 0
#         for generatorInd in range(n-k): # for all generators
#             g = checkMatrix[generatorInd, :] # get the genrator
            
#             string_format = '{:0>' + str(getCardin(g)) + '}'
#             for errorIndex in range(2**getCardin(g)): # for all errors with the support of that generator
#                 thisError = np.copy(g)
#                 modifyError = list(string_format.format("{:b}".format(errorIndex)))
#                 modifyError = list(map(int, modifyError))         
#                 thisError[thisError != 0] = (thisError[thisError != 0] + modifyError)%2 # get the error
                
#                 syndr_new = (syndr + getSyndromeFromError(getGenerator(thisError)))%2 # update to syndrome to check weight
                
#                 if(getCardin(thisError) == 0): continue
#                 thisMetric = (getCardin(syndr) - getCardin(syndr_new))/getCardin(thisError)
# #                 print('This syndrome = ' + str(syndr) + ', Syndrome if updated = ' + str(syndr_new) + ', before = ' + str(getCardin(syndr)) + ', after = ' + str(getCardin(syndr_new)) + ', This error = ' + str(thisError))
# #                 print('This generator ID = ' + str(generatorInd) + ', This error = ' + str(thisError) + ', thisMetric = ' + str(thisMetric))
#                 if(thisMetric > maxMetric):
#                     bestError = thisError
#                     maxMetric = thisMetric
#         print('Max metric = ' + str(maxMetric))
#         if(maxMetric != 0):
#             finalError = bestError
#             syndr = (syndr + getSyndromeFromError(getGenerator(bestError)))%2
#         if(maxMetric == 0):
#             if(getCardin(syndr) != 0): success = 0
#             else: success = 1
#             break
    
#     finalErrorModified = np.zeros(n, dtype=int)
#     finalErrorModified[(finalError[:n] == finalError[n:]) & (finalError[n:] == 1)] = 2
#     finalErrorModified[(finalError[:n] == 1) & (finalErrorModified != 2)] = 1
#     finalErrorModified[(finalError[n:] == 1) & (finalErrorModified != 2)] = 3
#     return finalErrorModified.flatten(), success

In [445]:
def getCardin(myVector):
    return np.sum(myVector != 0)

def getErrorFromSyndrome(syndr):
    success = 0
    finalError = np.zeros(2*errCheckRowModified.shape[0])
    while(getCardin(syndr) != 0):
        maxMetric = 0
        for generatorInd in range(n-k): # for all generators
            g = checkMatrix[generatorInd, :] # get the genrator
            g_modified = np.zeros(n, dtype=int)
            g_modified[(g[:n] == g[n:]) & (g[n:] == 1)] = 2
            g_modified[(g[:n] == 1) & (g_modified != 2)] = 1
            g_modified[(g[n:] == 1) & (g_modified != 2)] = 3
            
            string_format = '{:0>' + str(2*getCardin(g_modified)) + '}'
            for errorIndex in range(2**(2*getCardin(g_modified))): # for all errors with the support of that generator
                if(errorIndex == 0): continue
                thisError = np.copy(g_modified)
                
                modifyError = list(string_format.format("{:b}".format(errorIndex)))
                modifyError =  np.asarray(list(map(int, modifyError)) )
                
                temp_n = getCardin(g_modified)
                modifyErrorModified = np.zeros(temp_n, dtype=int)
                modifyErrorModified[(modifyError[:temp_n] == modifyError[temp_n:]) & (modifyError[temp_n:] == 1)] = 2
                modifyErrorModified[(modifyError[:temp_n] == 1) & (modifyErrorModified != 2)] = 1
                modifyErrorModified[(modifyError[temp_n:] == 1) & (modifyErrorModified != 2)] = 3
                
                thisError[thisError != 0] = modifyErrorModified # get the error
                
                           
                thisError1 = np.copy(thisError)
                thisError1[thisError == 1] = 1
                thisError1[thisError == 2] = 1
                thisError1[thisError == 3] = 0
                
                thisError2 = np.copy(thisError)
                thisError2[thisError == 1] = 0
                thisError2[thisError == 2] = 1
                thisError2[thisError == 3] = 1
                
                thisError = np.append(thisError1, thisError2)
                
                syndr_new = (syndr + getSyndromeFromError(getGenerator(thisError)))%2 # update to syndrome to check weight
                thisMetric = (getCardin(syndr) - getCardin(syndr_new))/getCardin(modifyErrorModified) # get the metric
                
                if(thisMetric > maxMetric):
                    bestError = thisError
                    maxMetric = thisMetric
#                 if(thisMetric == maxMetric):
#                     print('Error = ' + str(thisError) + ', |s_i+1| = ' + str(getCardin(syndr_new)) + ', |s_i| = ' + str(getCardin(syndr)) + ', |e| = ' + str(getCardin(thisError)))

        if(maxMetric != 0):
            finalError = bestError
            syndr = (syndr + getSyndromeFromError(getGenerator(bestError)))%2
        if(maxMetric == 0):
            break        
#         print('Max metric = ' + str(maxMetric) + ', Best error = ' + str(bestError) + ', Syndrome = ' + str(syndr))

    if(getCardin(syndr) != 0): success = 0
    else: success = 1

    finalErrorModified = np.zeros(n, dtype=int)
    finalErrorModified[(finalError[:n] == finalError[n:]) & (finalError[n:] == 1)] = 2
    finalErrorModified[(finalError[:n] == 1) & (finalErrorModified != 2)] = 1
    finalErrorModified[(finalError[n:] == 1) & (finalErrorModified != 2)] = 3
    return finalErrorModified.flatten(), success

In [490]:
p_xyz = 0.15 # p_xyz < 1/3
p_channel = [1-3*p_xyz, p_xyz, p_xyz, p_xyz] 
errMatrix = np.random.multinomial(1, p_channel, size=n)

errCheckRowModified = errMatrix@np.array([0,1,2,3])
channel_error = NKronModified(errCheckRowModified) # channel error
print('Error = ' + str(errCheckRowModified))

syndr = getSyndromeFromError(channel_error).flatten() # syndrome
print('Syndrome = ' + str(syndr))

print(' ')
recov = getErrorFromSyndrome(syndr) # recovered error
print(' ')
print('Recovered Error = ' + str(recov[0]))
print('Recovery Success = ' + str(recov[1] == 1))

Error = [0 0 0 1 1]
Syndrome = [0 1 0 1]
 
 
Recovered Error = [0 3 0 0 0]
Recovery Success = True


In [139]:
n-k

4

## Run the above two functions

In [117]:
checkMatrix


array([[1, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 1, 0, 1, 0, 0, 0, 1]])

In [194]:


# syndrome lookup table
# def getErrorFromSyndrome(syndr):
#     print('Syndrome before = ' + str(syndr))
#     recov = np.zeros(2*errCheckRowModified.shape[0])
# #     recov = np.array([0, 1, 0, 1, 0, 0, 0, 1, 1, 0])
#     for ind in range(n-k):
#         g = checkMatrix[ind,:]
#         string_format = '{:0>' + str(getCardin(g)) + '}'
        
#         maxValue = 0
#         for myIndex in range(2**getCardin(g)):
#             myError = np.copy(g)

#             flipArray = list(string_format.format("{:b}".format(myIndex)))
#             flipArray = list(map(int, flipArray))

#             myError[myError != 0] = (myError[myError != 0] + flipArray)%2

#             syndri = (syndr + getSyndromeFromError(getGenerator(myError)))%2
            
#             thisValue = (getCardin(syndr) - getCardin(syndri))/getCardin(myError)
#             if(thisValue > maxValue):
#                 maxError = myError
#                 maxValue = thisValue
#         print(ind)        
#         if(maxValue != 0):
#             print('Max error = ' + str(maxError))
#             print('Syndrome of error = ' + str(getSyndromeFromError(getGenerator(maxError))))
#             print('Old syndrome = ' + str(syndr))
#             syndr = (syndr + getSyndromeFromError(getGenerator(maxError)))%2
#             print('New syndrome = ' + str(syndr))
#             recov = (recov + maxError)%2
# #         elif(getCardin(syndr) == 0):
# #             success = 1
# #             break
    
#     print('Syndrome after = ' + str(syndr))
#     recovModified = np.zeros(n, dtype=int)
#     recovModified[(recov[:n] == recov[n:]) & (recov[n:] == 1)] = 2
#     recovModified[(recov[:n] == 1) & (recovModified != 2)] = 1
#     recovModified[(recov[n:] == 1) & (recovModified != 2)] = 3
#     return recovModified.flatten()

In [195]:
# # syndrome lookup table
# def getErrorFromSyndrome(syndr):
#     print(syndr)
#     recov = np.zeros(2*errCheckRowModified.shape[0])
# #     recov = np.array([0, 1, 0, 1, 0, 0, 0, 1, 1, 0])
#     for ind in range(n-k):
#         g = checkMatrix[ind,:]
#         string_format = '{:0>' + str(getCardin(g)) + '}'
        
#         maxValue = 0
#         for myIndex in range(2**getCardin(g)):
#             myError = np.copy(g)

#             flipArray = list(string_format.format("{:b}".format(myIndex)))
#             flipArray = list(map(int, flipArray))

#             myError[myError != 0] = (myError[myError != 0] + flipArray)%2

#             syndri = (syndr + getSyndromeFromError(getGenerator(myError)))%2
            
#             thisValue = (getCardin(syndr) - getCardin(syndri))/getCardin(myError)
#             if(thisValue > maxValue):
#                 maxError = myError
#                 maxValue = thisValue
#         print(' ')        
#         if(maxValue != 0):
#             print('Max error = ' + str(maxError))
#             print('Syndrome of error = ' + str(getSyndromeFromError(getGenerator(maxError))))
#             print('Old syndrome = ' + str(syndr))
#             syndr = (syndr + getSyndromeFromError(getGenerator(maxError)))%2
#             print('New syndrome = ' + str(syndr))
#             recov = (recov + maxError)%2
#         if(getCardin(syndr) == 0):
#             break
    
#     print(syndr)
#     recovModified = np.zeros(n, dtype=int)
#     recovModified[(recov[:n] == recov[n:]) & (recov[n:] == 1)] = 2
#     recovModified[(recov[:n] == 1) & (recovModified != 2)] = 1
#     recovModified[(recov[n:] == 1) & (recovModified != 2)] = 3
#     return recovModified.flatten()